<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-datasets" data-toc-modified-id="Load-datasets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load datasets</a></span></li><li><span><a href="#Data-preprocessing" data-toc-modified-id="Data-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data preprocessing</a></span><ul class="toc-item"><li><span><a href="#'sales_weather'-dataset-preprocessing" data-toc-modified-id="'sales_weather'-dataset-preprocessing-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>'sales_weather' dataset preprocessing</a></span></li><li><span><a href="#'beef_df'-preprocessing" data-toc-modified-id="'beef_df'-preprocessing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>'beef_df' preprocessing</a></span></li><li><span><a href="#'pork_df'-preprocessing" data-toc-modified-id="'pork_df'-preprocessing-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>'pork_df' preprocessing</a></span></li></ul></li></ul></div>

# Load datasets

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data/woochuri/')

In [115]:
# 우추리 일별 매출 데이터
sales = pd.read_excel('./woochuri_sales.xlsx')
sales['datetime'] = pd.Series(pd.date_range(start='2013-01-07',
                                           end='2020-12-31',
                                           freq='D'))
cols = ['datetime', 'sales', 'remark']
sales = sales[cols]
sales = sales.rename(columns={'datetime':'시간', 'sales':'일매출'})

# 일별 날씨 데이터 
weather = pd.read_csv('past_weather.csv', index_col=[0])
weather['시간'] = pd.to_datetime(weather['시간'])

# 두 데이터 merge
sales_weather = sales.merge(weather, how='inner', on='시간')
sales_weather.shape

(2916, 15)

In [84]:
# 소, 돼지 지역별 가격 데이터 각각 로드
import pymysql
db = pymysql.connect(host='localhost', user='younghun',
                    password='watson1259', db='beef_pork_db',
                    charset='utf8')
cursor = db.cursor()

beef_sql = "SELECT * FROM beef_prices"
pork_sql = "SELECT * FROM pork_prices"

beef_df = pd.read_sql(beef_sql, db)
pork_df = pd.read_sql(pork_sql, db)

beef_df.shape, pork_df.shape

((14647, 5), (12619, 5))

- 지역별 가격 중 한우, 육우별로 수도권/중부권/전국 지역 중 어떤 것을 선택할지 도메인 지식 전문가에게 물어보기

# Data preprocessing
## 'sales_weather' dataset preprocessing

In [5]:
# check dtype of features
sales_weather.dtypes

시간          datetime64[ns]
일매출                  int64
remark              object
지역                  object
평균기온               float64
최저기온               float64
최고기온               float64
1시간최다강수량           float64
일강수량               float64
평균풍속               float64
최대풍속               float64
평균상대습도             float64
최소상대습도               int64
1시간최다일사량           float64
일사량                float64
dtype: object

In [6]:
# check missing values of features
sales_weather.isnull().sum()

시간             0
일매출            0
remark      2822
지역             0
평균기온           0
최저기온           0
최고기온           0
1시간최다강수량    2202
일강수량        1733
평균풍속           2
최대풍속           0
평균상대습도         1
최소상대습도         0
1시간최다일사량       7
일사량            7
dtype: int64

In [7]:
# replace NaN values with zero value excluding with 'remark' feature
sales_weather[['1시간최다강수량','일강수량','평균풍속','평균상대습도','1시간최다일사량','일사량']] = sales_weather[['1시간최다강수량','일강수량','평균풍속','평균상대습도','1시간최다일사량','일사량']].fillna(value=0)
sales_weather.isnull().sum()

시간             0
일매출            0
remark      2822
지역             0
평균기온           0
최저기온           0
최고기온           0
1시간최다강수량       0
일강수량           0
평균풍속           0
최대풍속           0
평균상대습도         0
최소상대습도         0
1시간최다일사량       0
일사량            0
dtype: int64

## 'beef_df' preprocessing
- 소 지역별 가격 데이터
- 요일별로 나누어야 함
- 주의해야 할 점
    * 주말(토, 일)같은 경우는 빠져 있음
        * 그러므로 금요일 가격으로 대체하는 것이 좋을듯!
    * 지역값이 칼럼으로 가도록 Pivot 시키기

In [85]:
# 대전은 중부권이기 떄문에 중부권 가격만 추출
beef_df = beef_df[beef_df['지역'] == '중부권']
hanwoo_df = beef_df[beef_df['종류'] == '한우']
yukwoo_df = beef_df[beef_df['종류'] == '육우']

hanwoo_df = hanwoo_df.rename(columns={'종류': '한우', '가격': '한우가격'})
yukwoo_df = yukwoo_df.rename(columns={'종류': '육우', '가격': '육우가격'})
common_cols = ['날짜', '동물', '지역']
beef_df = hanwoo_df.merge(yukwoo_df, how='inner', on=common_cols)
print('중복 제거하기 전 shape:', beef_df.shape)
unique_beef_df = beef_df.drop_duplicates()
print('중족 제거 후 shape:', unique_beef_df.shape)

중복 제거하기 전 shape: (3493, 7)
중족 제거 후 shape: (1907, 7)


In [92]:
raw_date_df = pd.DataFrame(pd.date_range(start='2013-01-07', end='2020-12-31', freq='D'))
raw_date_df = raw_date_df.rename(columns={0: '날짜'})
merge_beef_df = raw_date_df.merge(unique_beef_df, how='left', on='날짜')
# 데이터가 없는 날짜는 직전 날의 가격으로 대체
merge_beef_df = merge_beef_df.fillna(method='ffill')

## 'pork_df' preprocessing

In [94]:
# 도메인 지식 상 시중에 판매되는 돼지고기 중 96.2%가 돼지 탕박임 -> 따라서 돼지 박피 데이터는 제거
# 또한 대전은 중부권이긴 하지만 지역의 종류에는 수도권, 영남권, 전국 밖에 없으므로 전국 값만 추출
cond_tang = pork_df['종류'] == '탕박'
cond_n = pork_df['지역'] == '전국'
pork_df = pork_df[(cond_tang)&(cond_n)]
pork_df = pork_df.drop_duplicates()

In [100]:
# 원래 범위의 날짜로 이루어진 데이터 프레임 만들고 이를 기준으로 left join 시키기
raw_date_df = pd.DataFrame(pd.date_range(start='2013-01-07', end='2020-12-31', freq='D'))
raw_date_df = raw_date_df.rename(columns={0: '날짜'})
merge_pork_df = raw_date_df.merge(pork_df, how='left', on='날짜')
merge_pork_df = merge_pork_date_df.fillna(method='ffill')
merge_pork_df = merge_pork_df.rename(columns={'가격': '돼지탕박가격'})

In [101]:
print('소 가격 데이터 shape:', merge_beef_df.shape)
print('돼지 가격 데이터 shape:', merge_pork_date_df.shape)
print(len(pd.date_range(start='2013-01-07', end='2020-12-31', freq='D')))

소 가격 데이터 shape: (2916, 7)
돼지 가격 데이터 shape: (2916, 5)
2916


In [104]:
# 한우, 육우, 돼지탕박 일자별 가격 병합한 데이터셋 만들기
merge_df = merge_beef_df.merge(merge_pork_df, how='inner', on='날짜')
columns = ['날짜', '한우가격', '육우가격', '돼지탕박가격']
beef_pork_df = merge_df[columns]
print('한우&육우&돼지탕박 일자별 가격 데이터프레임 shape:', beef_pork_df.shape)
beef_pork_df.head()

한우&육우&돼지탕박 일자별 가격 데이터프레임 shape: (2916, 4)


,날짜,한우가격,육우가격,돼지탕박가격
0,2013-01-07,14548.0,9169.0,3140.0
1,2013-01-08,15082.0,8105.0,3024.0
2,2013-01-09,15030.0,9669.0,3096.0
3,2013-01-10,15229.0,9674.0,3000.0
4,2013-01-11,14920.0,8286.0,2963.0


---

In [116]:
# remark의 결측치값은 모두 평일로 대체
sales['remark'] = sales['remark'].fillna('평일')

In [153]:
import holidays
from datetime import date
kor_holidays = holidays.Korea(years=[d for d in range(2013, 2021)])

holidays_dates = []
holidays_names = []
for date, name in kor_holidays.items():
    holidays_dates.append(date)
    holidays_names.append(name)
# 2013년 1월 7일부터이기 때문에 2013년 1월 7일 이전 공휴일은 설날 밖에없기 때문에 가장 첫 번째 요소 제거
holidays_df = pd.DataFrame({'공휴일날짜': holidays_dates,
                            '공휴일명': holidays_names}).sort_values(by='공휴일날짜')
holidays_df['공휴일날짜'] = pd.to_datetime(holidays_df['공휴일날짜'])

# sales(매출) 데이터로 날짜 기준으로 outer join 시키기
# 병합시킬 때 공휴일 데이터프레임 가장 첫 번째 행은 제외. 왜냐하면 2013년 1월 7일부터 데이터가 시작이기 때문!
sales_holiday_df = sales.merge(holidays_df.iloc[1:], how='outer', left_on='시간', right_on='공휴일날짜')

In [154]:
# loop돌면서 공휴일 직전으로 몇 일인지 파생변수 만들기 -> 어떻게 만들어볼까..?
sales_holiday_df.head(100)

,시간,일매출,remark,공휴일날짜,공휴일명
0,2013-01-07,422400,평일,NaT,NaN
1,2013-01-08,461900,평일,NaT,NaN
2,2013-01-09,492500,평일,NaT,NaN
3,2013-01-10,701300,평일,NaT,NaN
4,2013-01-11,1035000,평일,NaT,NaN
5,2013-01-12,1168600,평일,NaT,NaN
6,2013-01-13,696000,평일,NaT,NaN
7,2013-01-14,542000,평일,NaT,NaN
8,2013-01-15,503500,평일,NaT,NaN
9,2013-01-16,360500,평일,NaT,NaN
